In [1]:
#Carga de Librerias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import os

In [2]:
end_inv_2016 = pd.read_csv("EndInvFINAL12312016.csv")

In [4]:
#informacion del dataframe

end_inv_2016.head(), end_inv_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224489 entries, 0 to 224488
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InventoryId  224489 non-null  object 
 1   Store        224489 non-null  int64  
 2   City         223205 non-null  object 
 3   Brand        224489 non-null  int64  
 4   Description  224489 non-null  object 
 5   Size         224489 non-null  object 
 6   onHand       224489 non-null  int64  
 7   Price        224489 non-null  float64
 8   endDate      224489 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 15.4+ MB


(         InventoryId  Store          City  Brand                  Description  \
 0  1_HARDERSFIELD_58      1  HARDERSFIELD     58  Gekkeikan Black & Gold Sake   
 1  1_HARDERSFIELD_62      1  HARDERSFIELD     62     Herradura Silver Tequila   
 2  1_HARDERSFIELD_63      1  HARDERSFIELD     63   Herradura Reposado Tequila   
 3  1_HARDERSFIELD_72      1  HARDERSFIELD     72         No. 3 London Dry Gin   
 4  1_HARDERSFIELD_75      1  HARDERSFIELD     75    Three Olives Tomato Vodka   
 
     Size  onHand  Price     endDate  
 0  750mL      11  12.99  2016-12-31  
 1  750mL       7  36.99  2016-12-31  
 2  750mL       7  38.99  2016-12-31  
 3  750mL       4  34.99  2016-12-31  
 4  750mL       7  14.99  2016-12-31  ,
 None)

In [18]:
#Estadisticas de cada columna
for i in end_inv_2016.columns:
    print(i)
    #Esto imprime las estadísticas descriptivas (como count, mean, std, min, 25%, 50%, 75%, max) para esa columna.
    print(end_inv_2016[i].describe())
    print('--------------------------------------------------------------')

InventoryId
count                220667
unique               220667
top       81_PEMBROKE_90604
freq                      1
Name: InventoryId, dtype: object
--------------------------------------------------------------
Store
count    220667.000000
mean         43.523046
std          23.312948
min           1.000000
25%          23.000000
50%          44.000000
75%          66.000000
max          81.000000
Name: Store, dtype: float64
--------------------------------------------------------------
City
count        219433
unique           67
top       MOUNTMEND
freq          17848
Name: City, dtype: object
--------------------------------------------------------------
Brand
count    220667.000000
mean      14549.677999
std       13106.154970
min          58.000000
25%        3877.000000
50%        8368.000000
75%       24145.000000
max       90631.000000
Name: Brand, dtype: float64
--------------------------------------------------------------
Description
count                   220667
u

In [14]:
#Convertir la columna StartDate en tipo fecha

end_inv_2016['endDate'] = pd.to_datetime(end_inv_2016['endDate'])

In [10]:
print(end_inv_2016['Size'].unique())

['750mL' 'Liter' '750mL + 2/' '1.75L' '50mL' '375mL' '50mL 4 Pk'
 '200mL 3 Pk' '50mL 3 Pk' '750mL + 3/' '375mL 2 Pk' '100mL 4 Pk' '250mL'
 '100mL' '50mL 5 Pk' '3/100mL' '200mL 4 Pk' '250mL 4 Pk' '200mL' '1.5L'
 '5L' '4L' '187mL 4 Pk' '3L' '5.0 Oz' '187mL 3 Pk' '187mL' '500mL'
 '750mL 3 Pk' '400mL' '300mL' '330mL' '18L' '750mL 2 Pk' '5/2 oz' '720mL'
 '375mL 3 Pk' '750mL + 1/' '180mL' '200mL 5 Pk' '6L' '187mL 2 Pk'
 '750mL 4 Pk' '750mL 6 Pk' '750mL + 4/' '25.0' '128.0 Gal']


In [11]:
import re

def convertir_a_ml(size):
    # Expresión regular para extraer el número principal, unidad y posibles cantidades de paquetes
    match = re.match(r"(\d+(\.\d+)?)\s*(mL|L|liter|oz)?(?:\s*(\d+)?\s*Pk)?", str(size), re.IGNORECASE)
    
    if match:
        valor = float(match.group(1))  # Valor numérico principal
        unidad = match.group(3).lower() if match.group(3) else ''  # Unidad de medida
        cantidad_pk = int(match.group(4)) if match.group(4) else 1  # Cantidad en el paquete, por defecto 1
        
        # Convertir a mililitros según la unidad
        if unidad in ['l', 'liter']:
            return valor * 1000 * cantidad_pk  # Convertir litros a mililitros y multiplicar por cantidad
        elif unidad == 'oz':
            return valor * 29.5735 * cantidad_pk  # Convertir onzas a mililitros y multiplicar por cantidad
        elif unidad == 'ml':
            return valor * cantidad_pk  # Ya está en mililitros, multiplicar por cantidad
    return None

# Aplicar la función a la columna 'Size' para crear la columna 'Size_ml' en mL
end_inv_2016['Size_ml'] = end_inv_2016['Size'].apply(convertir_a_ml)

# Verificar el resultado
print(end_inv_2016[['Size', 'Size_ml']].head(10))

         Size  Size_ml
0       750mL    750.0
1       750mL    750.0
2       750mL    750.0
3       750mL    750.0
4       750mL    750.0
5       750mL    750.0
6       750mL    750.0
7       Liter      NaN
8       Liter      NaN
9  750mL + 2/    750.0


In [12]:
# Eliminar filas donde 'Size_ml' tiene valores NaN
end_inv_2016 = end_inv_2016.dropna(subset=['Size_ml'])

# Verificar el resultado
print(end_inv_2016['Size_ml'].unique())

[  750.      1750.        50.       375.       200.       600.
   150.       400.       250.       100.       800.      1000.
  1500.      5000.      4000.       748.      3000.       147.8675
   561.       187.       500.      2250.       300.       330.
 18000.       720.      1125.       180.      6000.       374.
  4500.    ]


In [17]:
#Borrar la columna Size para que solo quede Size_ml
end_inv_2016.drop('Size', axis=1)

,InventoryId,Store,City,Brand,Description,onHand,Price,endDate,Size_ml
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,11,12.99,2016-12-31,750.0
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,7,36.99,2016-12-31,750.0
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,7,38.99,2016-12-31,750.0
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,4,34.99,2016-12-31,750.0
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,7,14.99,2016-12-31,750.0
...,...,...,...,...,...,...,...,...,...
224484,81_PEMBROKE_90087,81,PEMBROKE,90087,Ch Mouton Rothschild 12 Paui,3,469.99,2016-12-31,750.0
224485,81_PEMBROKE_90088,81,PEMBROKE,90088,Ch Le Petite Mouton 12 Pauil,3,134.99,2016-12-31,750.0
224486,81_PEMBROKE_90089,81,PEMBROKE,90089,Ch Lynch Bages 12 Pauillac,3,119.99,2016-12-31,750.0
224487,81_PEMBROKE_90090,81,PEMBROKE,90090,Ch Lafite Rothschild 12,3,649.99,2016-12-31,750.0


In [21]:
del end_inv_2016['Size']
print(end_inv_2016.columns)

Index(['InventoryId', 'Store', 'City', 'Brand', 'Description', 'onHand',
       'Price', 'endDate', 'Size_ml'],
      dtype='object')


In [22]:
end_inv_2016.to_csv("End_inv_2016.csv")